# Pac-Man A* Pathfinding — Full Colab Notebook

# Packman

In [17]:
# STYLE 2: Classic Pac-Man theme
!pip install --quiet pillow imageio

from PIL import Image, ImageDraw, ImageFont
import imageio, os, math, heapq, random
from IPython.display import HTML, display
from base64 import b64encode

# ---------- Colors & Style ----------
BG = (0,0,0)           # black background
WALL = (0,130,255)     # neon blue walls
PELLET = (255,255,255)
PAC = (255, 200, 0)
GHOST1 = (255, 0, 0)   # blinky (red)
GHOST2 = (255, 184, 255) # pinky (pink)
TEXT = (255,255,255)
PROG_BG = (40,40,40)
PROG_FG = (255,215,0)

# ---------- Maze (same hybrid layout) ----------
ROWS = COLS = 24
MAZE = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1],
[1,0,1,0,1,0,1,1,1,0,1,0,1,1,1,0,1,0,1,0,1,0,0,1],
[1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,1],
[1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
]

CELL = 32
W, H = COLS*CELL, ROWS*CELL
os.makedirs('frames_style2', exist_ok=True)

# ---------- utilities & A* (same as style1) ----------
def in_bounds(pos): r,c = pos; return 0 <= r < ROWS and 0 <= c < COLS
def passable(pos): r,c = pos; return in_bounds(pos) and MAZE[r][c] == 0
def neighbors(pos):
    r,c = pos
    result = [(r+1,c),(r-1,c),(r,c+1),(r,c-1)]
    return [p for p in result if passable(p)]
def heuristic(a,b): return abs(a[0]-b[0])+abs(a[1]-b[1])
def a_star(start, goal, ghosts=set(), predicted=set()):
    if start==goal: return []
    frontier = []
    heapq.heappush(frontier,(0,start))
    came = {start:None}; cost = {start:0}
    while frontier:
        _,cur = heapq.heappop(frontier)
        if cur==goal: break
        for nxt in neighbors(cur):
            if nxt in ghosts or nxt in predicted: continue
            nc = cost[cur]+1
            for g in ghosts.union(predicted):
                if abs(nxt[0]-g[0])+abs(nxt[1]-g[1])==1: nc += 6
            if nxt not in cost or nc < cost[nxt]:
                cost[nxt]=nc
                heapq.heappush(frontier,(nc+heuristic(nxt,goal),nxt))
                came[nxt]=cur
    if goal not in came: return []
    path=[]; cur=goal
    while cur!=start:
        path.append(cur); cur=came[cur]
    path.reverse(); return path

# ---------- init ----------
pac = (1,1)
ghosts = [(3,10),(10,12)]
food = set((r,c) for r in range(ROWS) for c in range(COLS) if MAZE[r][c]==0)
if pac in food: food.remove(pac)
TOTAL_FOOD = len(food)
step = 0; max_steps = 1500

try: font = ImageFont.truetype("DejaVuSans.ttf",18)
except: font = ImageFont.load_default()

def render(pac, ghosts, food, path, step):
    img = Image.new("RGB",(W,H),BG)
    draw = ImageDraw.Draw(img)
    for r in range(ROWS):
        for c in range(COLS):
            x,y = c*CELL, r*CELL
            draw.rectangle([x,y,x+CELL-1,y+CELL-1], fill=BG)
            if MAZE[r][c]==1:
                draw.rectangle([x+2,y+2,x+CELL-3,y+CELL-3], fill=WALL)
    for (r,c) in food:
        cx,cy = c*CELL+CELL//2, r*CELL+CELL//2
        draw.ellipse([cx-3,cy-3,cx+3,cy+3], fill=PELLET)
    # path
    for (r,c) in path:
        x,y = c*CELL, r*CELL
        draw.rectangle([x+CELL//4,y+CELL//4,x+CELL-CELL//4,y+CELL-CELL//4], outline=(0,255,0))
    # pac
    pr,pc = pac; cx,cy = pc*CELL+CELL//2, pr*CELL+CELL//2
    draw.ellipse([cx-CELL//2+4,cy-CELL//2+4,cx+CELL//2-4,cy+CELL//2-4], fill=PAC)
    # ghosts
    for i,g in enumerate(ghosts):
        gr,gc = g; gx,gy = gc*CELL+CELL//2, gr*CELL+CELL//2
        color = GHOST1 if i==0 else GHOST2
        draw.ellipse([gx-CELL//2+6,gy-CELL//2+6,gx+CELL//2-6,gy+CELL//2-6], fill=color)
    score = (TOTAL_FOOD - len(food))
    draw.text((8,8), f"Score: {score}/{TOTAL_FOOD}   Step: {step}   Food left: {len(food)}", fill=TEXT, font=font)
    # progress bar
    bw, bh = W-16, 14
    bx, by = 8, H-28
    draw.rectangle([bx,by,bx+bw,by+bh], fill=PROG_BG)
    pct = score / TOTAL_FOOD
    draw.rectangle([bx,by,bx+int(bw*pct),by+bh], fill=PROG_FG)
    return img

frames_dir = 'frames_style2'; os.makedirs(frames_dir, exist_ok=True)

# ---------- Loop ----------
while food and step < max_steps:
    target = min(food, key=lambda f: abs(f[0]-pac[0])+abs(f[1]-pac[1]))
    ghost_set = set(ghosts)
    predicted = []
    for g in ghosts:
        neigh = neighbors(g)
        if neigh:
            neigh.sort(key=lambda p: abs(p[0]-pac[0])+abs(p[1]-pac[1]))
            predicted.append(neigh[0])
    path = a_star(pac,target,ghosts=ghost_set,predicted=set(predicted))
    if not path:
        remaining = list(food)
        if not remaining: break
        target = random.choice(remaining)
        path = a_star(pac,target,ghosts=ghost_set,predicted=set(predicted))
        if not path: break
    pac = path[0]
    if pac in food: food.remove(pac)
    # ghosts
    new=[]
    for g in ghosts:
        n=neighbors(g)
        if not n: new.append(g); continue
        if random.random() < 0.6:
            n.sort(key=lambda p: abs(p[0]-pac[0])+abs(p[1]-pac[1])); new.append(n[0])
        else:
            new.append(random.choice(n))
    ghosts = new
    if pac in ghosts:
        print("Pacman caught at step", step); break
    img = render(pac, ghosts, food.copy(), path, step)
    img.save(os.path.join(frames_dir, f"frame_{step:04d}.png"))
    step += 1

# final and video
img = render(pac, ghosts, food.copy(), [], step)
img.save(os.path.join(frames_dir, f"frame_{step:04d}.png"))
fns = sorted([os.path.join(frames_dir,f) for f in os.listdir(frames_dir) if f.endswith('.png')])
images = [imageio.imread(fn) for fn in fns]
imageio.mimsave('pacman_demo_style2.mp4', images, fps=8)
mp4 = open('pacman_demo_style2.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display(HTML(f'<video width="640" height="480" controls autoplay><source src="{data_url}" type="video/mp4"></video><br><a download="pacman_demo_style2.mp4" href="{data_url}">Download video</a>'))
print("Done. pacman_demo_style2.mp4")


/tmp/ipython-input-3941488577.py:170: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(fn) for fn in fns]


Done. pacman_demo_style2.mp4


In [28]:
# VERSION 2: C + 3 (Emoji-style Pac-Man + Emoji-style ghosts)
!pip install --quiet pillow imageio

from PIL import Image, ImageDraw, ImageFont
import imageio, os, heapq, random, math, time
from IPython.display import HTML, display
from base64 import b64encode

# ---------- Style & Maze ----------
BG = (0,0,0)
WALL = (0,130,255)
PELLET = (255,255,255)
PAC_COLOR = (255,220,80)
GHOST_A = (80,220,255)
GHOST_B = (255,184,255)
TEXT = (255,255,255)
PROG_BG = (40,40,40)
PROG_FG = (255,215,0)

ROWS = COLS = 24
MAZE = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1],
[1,0,1,0,1,0,1,1,1,0,1,0,1,1,1,0,1,0,1,0,1,0,0,1],
[1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,1],
[1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
]

CELL = 32
W, H = COLS*CELL, ROWS*CELL
os.makedirs('frames_c3', exist_ok=True)

# ---------- Utilities & A* ----------
def pos_in_bounds(p): r,c = p; return 0<=r<ROWS and 0<=c<COLS
def passable(p): r,c = p; return pos_in_bounds(p) and MAZE[r][c]==0
def neighbors(p):
    r,c = p
    cand = [(r+1,c),(r-1,c),(r,c+1),(r,c-1)]
    return [x for x in cand if passable(x)]
def heuristic(a,b): return abs(a[0]-b[0])+abs(a[1]-b[1])

def a_star(start, goal, ghosts=set(), predicted=set()):
    if start==goal: return []
    frontier=[]; heapq.heappush(frontier,(0,start))
    came={start:None}; cost={start:0}
    while frontier:
        _,cur = heapq.heappop(frontier)
        if cur==goal: break
        for nxt in neighbors(cur):
            if nxt in ghosts or nxt in predicted: continue
            nc = cost[cur]+1
            for g in ghosts.union(predicted):
                if abs(nxt[0]-g[0])+abs(nxt[1]-g[1])==1:
                    nc += 6
            if nxt not in cost or nc < cost[nxt]:
                cost[nxt]=nc
                heapq.heappush(frontier,(nc+heuristic(nxt,goal),nxt))
                came[nxt]=cur
    if goal not in came: return []
    path=[]; cur=goal
    while cur!=start:
        path.append(cur); cur=came[cur]
    path.reverse(); return path

# ---------- Icon drawing helpers (emoji pac & ghosts) ----------
def draw_emoji_pac(size=64):
    img = Image.new("RGBA",(size,size),(0,0,0,0))
    d = ImageDraw.Draw(img)
    cx,cy = size//2, size//2
    r = int(size*0.42)
    # face circle
    d.ellipse([cx-r,cy-r,cx+r,cy+r], fill=PAC_COLOR)
    # mouth simple wedge
    wedge = [(cx,cy),(cx+int(r*1.0),cy-int(r*0.35)),(cx+int(r*1.0),cy+int(r*0.35))]
    d.polygon(wedge, fill=(0,0,0))
    # eye (rounded)
    ex = cx + int(r* -0.1); ey = cy - int(r*0.45)
    d.ellipse([ex-int(r*0.12),ey-int(r*0.12),ex+int(r*0.12),ey+int(r*0.12)], fill=(255,255,255))
    d.ellipse([ex-int(r*0.06)+3,ey-int(r*0.06),ex+int(r*0.06)+3,ey+int(r*0.06)], fill=(0,0,0))
    return img

def draw_emoji_ghost(size=64, color=(80,220,255)):
    img = Image.new("RGBA",(size,size),(0,0,0,0))
    d = ImageDraw.Draw(img)
    cx,cy = size//2, size//2 - 4
    r = int(size*0.38)
    d.ellipse([cx-r,cy-r,cx+r,cy+r], fill=color)
    bottom_y = cy + r
    wave_h = int(size*0.18)
    left = cx - r; right = cx + r
    segments = 6; seg_w = (right-left)/segments
    poly = [(left, cy)]
    for i in range(segments+1):
        x = left + i*seg_w
        y = bottom_y - (wave_h if i%2==0 else 0)
        poly.append((x,y))
    poly.append((right,cy))
    d.polygon(poly, fill=color)
    # eyes
    ex = cx - int(r*0.35); ey = cy - int(r*0.12)
    d.ellipse([ex-int(r*0.13),ey-int(r*0.13),ex+int(r*0.13),ey+int(r*0.13)], fill=(255,255,255))
    ex2 = cx + int(r*0.12)
    d.ellipse([ex2-int(r*0.13),ey-int(r*0.13),ex2+int(r*0.13),ey+int(r*0.13)], fill=(255,255,255))
    d.ellipse([ex-int(r*0.06)+2,ey-int(r*0.06),ex+int(r*0.02)+2,ey+int(r*0.02)], fill=(0,0,0))
    d.ellipse([ex2-int(r*0.06)+2,ey-int(r*0.06),ex2+int(r*0.02)+2,ey+int(r*0.02)], fill=(0,0,0))
    return img

# ---------- Init ----------
pac = (1,1)
ghosts = [(3,10),(10,12)]
food = set((r,c) for r in range(ROWS) for c in range(COLS) if MAZE[r][c]==0)
if pac in food: food.remove(pac)
TOTAL_FOOD = len(food)
STEP_LIMIT = 2000
PELLETS_TO_TRIGGER_CHASE = 5

PAC_ICON = draw_emoji_pac(64)
GHOST_ICON_A = draw_emoji_ghost(64, color=GHOST_A)
GHOST_ICON_B = draw_emoji_ghost(64, color=GHOST_B)

try: FONT = ImageFont.truetype("DejaVuSans.ttf",18)
except: FONT = ImageFont.load_default()

# ---------- Rendering ----------
os.makedirs('frames_c3', exist_ok=True)
def render_frame(pac, ghosts, food, path, step, score):
    img = Image.new("RGB",(W,H),BG)
    d = ImageDraw.Draw(img)
    for r in range(ROWS):
        for c in range(COLS):
            x,y = c*CELL, r*CELL
            d.rectangle([x,y,x+CELL-1,y+CELL-1], fill=BG)
            if MAZE[r][c]==1:
                d.rectangle([x+2,y+2,x+CELL-3,y+CELL-3], fill=WALL)
    for (r,c) in food:
        cx,cy = c*CELL+CELL//2, r*CELL+CELL//2
        d.ellipse([cx-3,cy-3,cx+3,cy+3], fill=PELLET)
    if path:
        for (r,c) in path:
            x,y = c*CELL, r*CELL
            d.rectangle([x+CELL//4,y+CELL//4,x+CELL-CELL//4,y+CELL-CELL//4], outline=(0,255,0))
    pr,pc = pac; px,py = pc*CELL+CELL//2, pr*CELL+CELL//2
    pi = PAC_ICON.resize((int(CELL*0.9), int(CELL*0.9)), Image.LANCZOS)
    img.paste(pi, (px - pi.width//2, py - pi.height//2), pi)
    for i,g in enumerate(ghosts):
        gr,gc = g; gx,gy = gc*CELL+CELL//2, gr*CELL+CELL//2
        gi = GHOST_ICON_A if i==0 else GHOST_ICON_B
        gi_s = gi.resize((int(CELL*0.9), int(CELL*0.9)), Image.LANCZOS)
        img.paste(gi_s, (gx - gi_s.width//2, gy - gi_s.height//2), gi_s)
    text = f"Score: {score}/{TOTAL_FOOD}   Step: {step}   Food left: {len(food)}"
    d.text((8,8), text, fill=TEXT, font=FONT)
    bx,by,bw,bh = 8, H-28, W-16, 14
    d.rectangle([bx,by,bx+bw,by+bh], fill=PROG_BG)
    pct = score / TOTAL_FOOD if TOTAL_FOOD>0 else 1.0
    d.rectangle([bx,by,bx+int(bw*pct),by+bh], fill=PROG_FG)
    return img

# ---------- Main Loop ----------
frames = []; step = 0; score = 0
while food and step < STEP_LIMIT:
    target = min(food, key=lambda f: abs(f[0]-pac[0])+abs(f[1]-pac[1]))
    ghost_set = set(ghosts)
    predicted = set()
    for g in ghosts:
        neigh = neighbors(g)
        if neigh:
            neigh.sort(key=lambda p: abs(p[0]-pac[0])+abs(p[1]-pac[1]))
            predicted.add(neigh[0])
    path = a_star(pac, target, ghosts=ghost_set, predicted=predicted)
    if not path:
        if not food: break
        target = random.choice(list(food))
        path = a_star(pac, target, ghosts=ghost_set, predicted=predicted)
        if not path: break
    pac = path[0]
    if pac in food:
        food.remove(pac); score += 1
    # ghost movement: switch to chase after threshold
    new_ghosts = []
    chase_mode = (score >= PELLETS_TO_TRIGGER_CHASE)
    for g in ghosts:
        if chase_mode:
            g_path = a_star(g, pac, ghosts=set(), predicted=set())
            if g_path:
                new_pos = g_path[0]
            else:
                choices = neighbors(g)
                new_pos = random.choice(choices) if choices else g
        else:
            choices = neighbors(g)
            if not choices:
                new_pos = g
            else:
                if random.random() < 0.6:
                    choices.sort(key=lambda p: abs(p[0]-pac[0])+abs(p[1]-pac[1])); new_pos = choices[0]
                else:
                    new_pos = random.choice(choices)
        new_ghosts.append(new_pos)
    ghosts = new_ghosts
    if pac in ghosts:
        img = render_frame(pac, ghosts, food.copy(), [], step, score)
        d = ImageDraw.Draw(img)
        overlay_text = "GAME OVER"
        w_text, h_text = d.textsize(overlay_text, font=FONT)
        d.rectangle([(W//2-160, H//2-40), (W//2+160, H//2+40)], fill=(0,0,0,180))
        d.text((W//2 - w_text//2, H//2 - h_text//2), overlay_text, fill=(255,0,0), font=FONT)
        fname = f"frames_c3/frame_{step:04d}.png"; img.save(fname); frames.append(fname)
        print("Pac-Man caught! Game Over.")
        break
    img = render_frame(pac, ghosts, food.copy(), path, step, score)
    fname = f"frames_c3/frame_{step:04d}.png"; img.save(fname); frames.append(fname)
    step += 1

if not food:
    img = render_frame(pac, ghosts, food.copy(), [], step, score)
    d = ImageDraw.Draw(img)
    overlay_text = f"CONGRATULATIONS! Score: {score}/{TOTAL_FOOD}"
    w_text, h_text = d.textsize(overlay_text, font=FONT)
    d.rectangle([(W//2-260, H//2-40), (W//2+260, H//2+40)], fill=(0,0,0,180))
    d.text((W//2 - w_text//2, H//2 - h_text//2), overlay_text, fill=(0,255,0), font=FONT)
    fname = f"frames_c3/frame_{step:04d}.png"; img.save(fname); frames.append(fname)

# assemble video
filenames = sorted(frames)
images = [imageio.imread(fn) for fn in filenames]
imageio.mimsave('pacman_demo_c3.mp4', images, fps=8)
mp4 = open('pacman_demo_c3.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
html = f'''<video width="640" height="480" controls autoplay>
    <source src="{data_url}" type="video/mp4"></video><br>
    <a download="pacman_demo_c3.mp4" href="{data_url}">Download video</a>'''
display(HTML(html))
print("Done. pacman_demo_c3.mp4")


/tmp/ipython-input-4268930796.py:244: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(fn) for fn in filenames]


Done. pacman_demo_c3.mp4


In [24]:
# Install dependencies (Colab)
!pip install --quiet pillow imageio

from PIL import Image, ImageDraw, ImageFont
import imageio, os, heapq, random, math
from IPython.display import HTML, display
from base64 import b64encode

# ---------- Style & Maze (your exact MAZE) ----------
BG = (0,0,0)
WALL = (0,130,255)
PELLET = (255,255,255)
PAC_COLOR = (255,220,80)
GHOST_A = (80,220,255)
GHOST_B = (255,184,255)
TEXT = (255,255,255)
PROG_BG = (40,40,40)
PROG_FG = (255,215,0)

ROWS = COLS = 24
MAZE = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1],
[1,0,1,0,1,0,1,1,1,0,1,0,1,1,1,0,1,0,1,0,1,0,0,1],
[1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,1],
[1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1],
[1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1],
[1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1],
[1,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,0,0,1],
[1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,1],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
]

CELL = 32
W, H = COLS*CELL, ROWS*CELL
os.makedirs('frames_c3', exist_ok=True)

# ---------- Utilities & A* ----------
def pos_in_bounds(p): r,c = p; return 0<=r<ROWS and 0<=c<COLS
def passable(p): r,c = p; return pos_in_bounds(p) and MAZE[r][c]==0
def neighbors(p):
    r,c = p
    cand = [(r+1,c),(r-1,c),(r,c+1),(r,c-1)]
    return [x for x in cand if passable(x)]
def heuristic(a,b): return abs(a[0]-b[0])+abs(a[1]-b[1])

def a_star(start, goal, ghosts=set(), predicted=set()):
    if start==goal: return []
    frontier=[]; heapq.heappush(frontier,(0,start))
    came={start:None}; cost={start:0}
    while frontier:
        _,cur = heapq.heappop(frontier)
        if cur==goal: break
        for nxt in neighbors(cur):
            if nxt in ghosts or nxt in predicted: continue
            nc = cost[cur]+1
            for g in ghosts.union(predicted):
                if abs(nxt[0]-g[0])+abs(nxt[1]-g[1])==1:
                    nc += 6
            if nxt not in cost or nc < cost[nxt]:
                cost[nxt]=nc
                heapq.heappush(frontier,(nc+heuristic(nxt,goal),nxt))
                came[nxt]=cur
    if goal not in came: return []
    path=[]; cur=goal
    while cur!=start:
        path.append(cur); cur=came[cur]
    path.reverse(); return path

# ---------- Icon drawing helpers (emoji pac & ghosts) ----------
def draw_emoji_pac(size=64):
    img = Image.new("RGBA",(size,size),(0,0,0,0))
    d = ImageDraw.Draw(img)
    cx,cy = size//2, size//2
    r = int(size*0.42)
    # face circle
    d.ellipse([cx-r,cy-r,cx+r,cy+r], fill=PAC_COLOR)
    # mouth simple wedge
    wedge = [(cx,cy),(cx+int(r*1.0),cy-int(r*0.35)),(cx+int(r*1.0),cy+int(r*0.35))]
    d.polygon(wedge, fill=(0,0,0))
    # eye (rounded)
    ex = cx + int(r* -0.1); ey = cy - int(r*0.45)
    d.ellipse([ex-int(r*0.12),ey-int(r*0.12),ex+int(r*0.12),ey+int(r*0.12)], fill=(255,255,255))
    d.ellipse([ex-int(r*0.06)+3,ey-int(r*0.06),ex+int(r*0.06)+3,ey+int(r*0.06)], fill=(0,0,0))
    return img

def draw_emoji_ghost(size=64, color=(80,220,255)):
    img = Image.new("RGBA",(size,size),(0,0,0,0))
    d = ImageDraw.Draw(img)
    cx,cy = size//2, size//2 - 4
    r = int(size*0.38)
    d.ellipse([cx-r,cy-r,cx+r,cy+r], fill=color)
    bottom_y = cy + r
    wave_h = int(size*0.18)
    left = cx - r; right = cx + r
    segments = 6; seg_w = (right-left)/segments
    poly = [(left, cy)]
    for i in range(segments+1):
        x = left + i*seg_w
        y = bottom_y - (wave_h if i%2==0 else 0)
        poly.append((x,y))
    poly.append((right,cy))
    d.polygon(poly, fill=color)
    # eyes
    ex = cx - int(r*0.35); ey = cy - int(r*0.12)
    d.ellipse([ex-int(r*0.13),ey-int(r*0.13),ex+int(r*0.13),ey+int(r*0.13)], fill=(255,255,255))
    ex2 = cx + int(r*0.12)
    d.ellipse([ex2-int(r*0.13),ey-int(r*0.13),ex2+int(r*0.13),ey+int(r*0.13)], fill=(255,255,255))
    d.ellipse([ex-int(r*0.06)+2,ey-int(r*0.06),ex+int(r*0.02)+2,ey+int(r*0.02)], fill=(0,0,0))
    d.ellipse([ex2-int(r*0.06)+2,ey-int(r*0.06),ex2+int(r*0.02)+2,ey+int(r*0.02)], fill=(0,0,0))
    return img

# ---------- Init ----------
pac = (1,1)
ghosts = [(3,10),(10,12)]
food = set((r,c) for r in range(ROWS) for c in range(COLS) if MAZE[r][c]==0)
if pac in food: food.remove(pac)
TOTAL_FOOD = len(food)
STEP_LIMIT = 2000
PELLETS_TO_TRIGGER_CHASE = 30

PAC_ICON = draw_emoji_pac(64)
GHOST_ICON_A = draw_emoji_ghost(64, color=GHOST_A)
GHOST_ICON_B = draw_emoji_ghost(64, color=GHOST_B)

try: FONT = ImageFont.truetype("DejaVuSans.ttf",18)
except: FONT = ImageFont.load_default()

# ---------- Rendering ----------
os.makedirs('frames_c3', exist_ok=True)
def render_frame(pac, ghosts, food, path, step, score):
    img = Image.new("RGB",(W,H),BG)
    d = ImageDraw.Draw(img)
    for r in range(ROWS):
        for c in range(COLS):
            x,y = c*CELL, r*CELL
            d.rectangle([x,y,x+CELL-1,y+CELL-1], fill=BG)
            if MAZE[r][c]==1:
                d.rectangle([x+2,y+2,x+CELL-3,y+CELL-3], fill=WALL)
    for (r,c) in food:
        cx,cy = c*CELL+CELL//2, r*CELL+CELL//2
        d.ellipse([cx-3,cy-3,cx+3,cy+3], fill=PELLET)
    if path:
        for (r,c) in path:
            x,y = c*CELL, r*CELL
            d.rectangle([x+CELL//4,y+CELL//4,x+CELL-CELL//4,y+CELL-CELL//4], outline=(0,255,0))
    pr,pc = pac; px,py = pc*CELL+CELL//2, pr*CELL+CELL//2
    pi = PAC_ICON.resize((int(CELL*0.9), int(CELL*0.9)), Image.LANCZOS)
    img.paste(pi, (px - pi.width//2, py - pi.height//2), pi)
    for i,g in enumerate(ghosts):
        gr,gc = g; gx,gy = gc*CELL+CELL//2, gr*CELL+CELL//2
        gi = GHOST_ICON_A if i==0 else GHOST_ICON_B
        gi_s = gi.resize((int(CELL*0.9), int(CELL*0.9)), Image.LANCZOS)
        img.paste(gi_s, (gx - gi_s.width//2, gy - gi_s.height//2), gi_s)
    text = f"Score: {score}/{TOTAL_FOOD}   Step: {step}   Food left: {len(food)}"
    d.text((8,8), text, fill=TEXT, font=FONT)
    bx,by,bw,bh = 8, H-28, W-16, 14
    d.rectangle([bx,by,bx+bw,by+bh], fill=PROG_BG)
    pct = score / TOTAL_FOOD if TOTAL_FOOD>0 else 1.0
    d.rectangle([bx,by,bx+int(bw*pct),by+bh], fill=PROG_FG)
    return img

# ---------- Main Loop ----------
frames = []; step = 0; score = 0
while food and step < STEP_LIMIT:
    target = min(food, key=lambda f: abs(f[0]-pac[0])+abs(f[1]-pac[1]))
    ghost_set = set(ghosts)
    predicted = set()
    for g in ghosts:
        neigh = neighbors(g)
        if neigh:
            neigh.sort(key=lambda p: abs(p[0]-pac[0])+abs(p[1]-pac[1]))
            predicted.add(neigh[0])
    path = a_star(pac, target, ghosts=ghost_set, predicted=predicted)
    if not path:
        if not food: break
        target = random.choice(list(food))
        path = a_star(pac, target, ghosts=ghost_set, predicted=predicted)
        if not path: break
    pac = path[0]
    if pac in food:
        food.remove(pac); score += 1
    # ghost movement: switch to chase after threshold
    new_ghosts = []
    chase_mode = (score >= PELLETS_TO_TRIGGER_CHASE)
    for g in ghosts:
        if chase_mode:
            g_path = a_star(g, pac, ghosts=set(), predicted=set())
            if g_path:
                new_pos = g_path[0]
            else:
                choices = neighbors(g)
                new_pos = random.choice(choices) if choices else g
        else:
            choices = neighbors(g)
            if not choices:
                new_pos = g
            else:
                if random.random() < 0.6:
                    choices.sort(key=lambda p: abs(p[0]-pac[0])+abs(p[1]-pac[1])); new_pos = choices[0]
                else:
                    new_pos = random.choice(choices)
        new_ghosts.append(new_pos)
    ghosts = new_ghosts
    if pac in ghosts:
        img = render_frame(pac, ghosts, food.copy(), [], step, score)
        d = ImageDraw.Draw(img)
        overlay_text = "GAME OVER"
        w_text, h_text = d.textsize(overlay_text, font=FONT)
        d.rectangle([(W//2-160, H//2-40), (W//2+160, H//2+40)], fill=(0,0,0,180))
        d.text((W//2 - w_text//2, H//2 - h_text//2), overlay_text, fill=(255,0,0), font=FONT)
        fname = f"frames_c3/frame_{step:04d}.png"; img.save(fname); frames.append(fname)
        print("Pac-Man caught! Game Over.")
        break
    img = render_frame(pac, ghosts, food.copy(), path, step, score)
    fname = f"frames_c3/frame_{step:04d}.png"; img.save(fname); frames.append(fname)
    step += 1

if not food:
    img = render_frame(pac, ghosts, food.copy(), [], step, score)
    d = ImageDraw.Draw(img)
    overlay_text = f"CONGRATULATIONS! Score: {score}/{TOTAL_FOOD}"
    w_text, h_text = d.textsize(overlay_text, font=FONT)
    d.rectangle([(W//2-260, H//2-40), (W//2+260, H//2+40)], fill=(0,0,0,180))
    d.text((W//2 - w_text//2, H//2 - h_text//2), overlay_text, fill=(0,255,0), font=FONT)
    fname = f"frames_c3/frame_{step:04d}.png"; img.save(fname); frames.append(fname)

# assemble video
filenames = sorted(frames)
images = [imageio.imread(fn) for fn in filenames]
imageio.mimsave('pacman_demo_c3.mp4', images, fps=8)
mp4 = open('pacman_demo_c3.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
html = f'''<video width="640" height="480" controls autoplay>
    <source src="{data_url}" type="video/mp4"></video><br>
    <a download="pacman_demo_c3.mp4" href="{data_url}">Download video</a>'''
display(HTML(html))
print("Done. pacman_demo_c3.mp4")


/tmp/ipython-input-885547438.py:250: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(fn) for fn in filenames]


Done. pacman_demo_c3.mp4
